# Optimize parameters of reduced morphology model 

Reduction method is Marasco method, 7 folding passes.

In [ ]:
# Wait until other job finishes
# import time
# expected_runtime_sec = 60.0 * 2.05 * 50.0 * 2.0
# time.sleep(expected_runtime_sec)

# Enable interactive plots with backend 'notebook'
%matplotlib inline

# Enable connecting with ipyton console --existing
# %connect_info

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd /home/luye/workspace/bgcellmodels/GilliesWillshaw/

# print code version (hash of checked out version)
!git log -1 # --format="%H"

## Notes for running optimization

To run parallel optimization (individuals in population evaluated over many cores), the ipyprallel module can be used. To use ipyparallel you must start a controller and a number of engine (worker) instances before starting ipython (see http://ipyparallel.readthedocs.io/en/latest/intro.html), e.g:

```bash
ipcluster start -n 6
ipengine --debug # only if you want to start a kernel with visible output
```

, where the -n argument is the number of workers / cores. Alternatively do

```bash
ipcontroller
ipengine --debug --log-to-file=True # start one engine with custom arguments
ipcluster engines --n=4 # start four more engines
```

Or alternatively do

```bash
ipython profile create --parallel --profile=myprofile # do this once
ipcluster --profile=myprofile # can also use profile with ipcontroller/ipengine commands
```
	
Make sure that Hoc can find the .hoc model files by either executing above command in the directory containing those files, or adding the relevant directories to the environment variable `$HOC_LIBRARY_PATH` (this could also be done in 
your protocol or cellmodel script using `os.environ["HOC_LIBRARY_PATH"]`)

## Initialization

In [ ]:
# Python standard library
import pickle, pprint
pp = pprint.PrettyPrinter(indent=2)
import numpy as np

# BluePyOpt modules
import bluepyopt as bpop
import bluepyopt.ephys as ephys

# Our custom BluePyOpt modules
from optimize.bpop_cellmodels import StnFullModel, StnReducedModel
from optimize.bpop_extensions import NrnScaleRangeParameter, NrnSeclistLocationExt
from optimize.bpop_protocols_stn import BpopProtocolWrapper
from optimize.bpop_optimize_stn import get_map_function

import optimize.bpop_features_stn as features_stn
import optimize.bpop_parameters_stn as parameters_stn
import optimize.bpop_protocols_stn as protocols_stn
from optimize import bpop_optimisation

# Gillies & Willshaw model mechanisms
import gillies_model
gleak_name = gillies_model.gleak_name

# Physiology parameters
from bgcellmodels.cellpopdata import StnModel
from evalmodel.proto_common import StimProtocol

CLAMP_PLATEAU = StimProtocol.CLAMP_PLATEAU
CLAMP_REBOUND = StimProtocol.CLAMP_REBOUND
MIN_SYN_BURST = StimProtocol.MIN_SYN_BURST
SYN_BACKGROUND_HIGH = StimProtocol.SYN_BACKGROUND_HIGH

In [ ]:
# Import our analysis modules
# %load_ext autoreload
# %autoreload 1
# %aimport optimize.bpop_analysis_stn
# bpop_analysis = optimize.bpop_analysis_stn

import optimize.bpop_analysis_stn
import optimize.bpop_analysis_pop
resp_analysis = optimize.bpop_analysis_stn
pop_analysis = optimize.bpop_analysis_pop

from optimize.bpop_analysis_stn import (
    run_proto_responses, plot_proto_responses, 
    save_proto_responses, load_proto_responses
)

In [ ]:
# Distributed logging
from bgcellmodels.common import stdutil, logutils
logger = logutils.getBasicLogger('stn_opt')
logutils.setLogLevel('quiet', ['marasco', 'folding', 'redops', 
                               'bluepyopt.ephys.parameters', 'bluepyopt.ephys.recordings', 
                               'bluepyopt.ephys.simulators'])
logutils.setLogLevel('verbose', ['bpop_ext', 'stn_protos', 'stn_opt', 'reduce_cell', 
                                 '__main__', 'bluepyopt.ephys.efeatures'])

# proto_logger = logging.getLogger('stn_protos')
# logutils.install_mp_handler(logger=proto_logger) # record some logs on multiple threads

In [ ]:
# Data storage for optimization
opt = stdutil.dotdict()

## Define Cell Model & Parameters

Optimize parameters of reduced model. Passive parameters should be fitted first and set in the module file. Parameters to optimize, stimulation protocols to use for evaluation (objective function), and electrophysiological features extracted from the protocol responses can be set in following modules:

```python
optimize.bpop_parameters_stn
optimize.bpop_protocols_stn
optimize.bpop_features_stn
```

In [ ]:
def make_parameters(self):
    """
    Make parameters for optimization
    
    @return    tuple(frozen_params, free_params) where each entry is a 
               list(ephys.parameters.NrnParameter)
               
    @note    must be called only once in incremental optimisation
    """
    # Define regions for applying parameters
    
    somatic_region = NrnSeclistLocationExt(
                        'somatic', 
                        seclist_name='somatic')
    
    dendritic_all = NrnSeclistLocationExt(
                        'dendritic', 
                        seclist_name='dendritic')
    
    dendritic_fold = NrnSeclistLocationExt(
                        'dendritic', 
                        seclist_name='dendritic',
                        secname_filter=r'zip') # only apply to folded sections
    
    # SOMATIC PASSIVE PARAMETERS
    
    soma_gl_factor = NrnScaleRangeParameter(
                        name='gleak_soma_scale',
                        param_name=gleak_name,
                        value = 1.0,
                        bounds=[0.05, 10.0],
                        locations=[somatic_region],
                        frozen=False)

    soma_cm_factor = NrnScaleRangeParameter(
                        name='cm_soma_scale',
                        param_name='cm',
                        value = 1.0,
                        bounds=[0.05, 10.0],
                        locations=[somatic_region],
                        frozen=False)
    
    soma_passive_params = [soma_gl_factor, soma_cm_factor]
    
    # TODO: Eleak, diam, L (passive/subtreshold optimization)
    
    # SOMATIC ACTIVE PARAMETERS
    MIN_SCALE_GBAR = 0.1
    MAX_SCALE_GBAR = 10.0
    
    # channels with significant densities in soma section
    gbar_active_soma = ['gna_NaL', 'gk_Ih', 'gk_sKCa', 'gcaN_HVA']
    
    soma_gbar_params = [
        
        NrnScaleRangeParameter(
                name		= gbar_name + '_soma_scale',
                param_name	= gbar_name,
                value		= 1.0,
                bounds		= [MIN_SCALE_GBAR, MAX_SCALE_GBAR],
                locations	= [somatic_region],
                frozen		= False)

        for gbar_name in gbar_active_soma
    ]
    
    # DENDRITIC PASSIVE PARAMETERS
    
    dend_gl_factor = NrnScaleRangeParameter(
                        name='gleak_dend_scale',
                        param_name=gleak_name,
                        value = 1.0,
                        bounds=[0.2, 5.0],
                        locations=[dendritic_fold],
                        frozen=False)

    dend_cm_factor = NrnScaleRangeParameter(
                        name='cm_dend_scale',
                        param_name='cm',
                        value = 1.0,
                        bounds=[0.2, 5.0],
                        locations=[dendritic_fold],
                        frozen=False)

    dend_ra_param = ephys.parameters.NrnSectionParameter(
                        name='Ra_dend',
                        param_name='Ra',
                        value = 150.224, # default in Gillies model
                        bounds=[50, 500.0],
                        locations=[dendritic_fold],
                        frozen=False)
    
    # TODO: Eleak, diam, L, gbar (passive/subtreshold optimization)
    
    dend_passive_params = [dend_gl_factor, dend_cm_factor, dend_ra_param]
    
    # DENDRITIC ACTIVE PARAMETERS
    
    scaled_gbar = ['gna_NaL', 'gk_Ih', 'gk_sKCa', 'gcaT_CaT', 'gcaL_HVA', 'gcaN_HVA']
    
    MIN_SCALE_GBAR = 0.1
    MAX_SCALE_GBAR = 10.0
    
    dend_gbar_params = [
        
        NrnScaleRangeParameter(
                name		= gbar_name + '_dend_scale',
                param_name	= gbar_name,
                value		= 1.0,
                bounds		= [MIN_SCALE_GBAR, MAX_SCALE_GBAR],
                locations	= [dendritic_all], # SETPARAM: set to dendritic_fold for incremental optimisation
                frozen		= False)

        for gbar_name in scaled_gbar
    ]
    
    # TODO: shift params of V-gated and Ca-gated gating functions
    
    # CALCIUM DYNAMICS PARAMETERS
    
    decay_orig = 1.857456645e+02 # see Cacum.mod
    ca_decay = ephys.parameters.NrnGlobalParameter(
                        name='Ca_decay',
                        param_name='buftau_Cacum',
                        value=decay_orig,
                        bounds=[decay_orig/5.0, decay_orig*5.0],
                        frozen=False)
    
    depth_orig = 200.0 # see Cacum.mod
    ca_depth = ephys.parameters.NrnGlobalParameter(
                        name='Ca_depth',
                        param_name='depth_Cacum',
                        value=depth_orig, # see Cacum.mod
                        bounds=[depth_orig/5.0, depth_orig*5.0],
                        frozen=False)
    
    all_ca_params = [ca_decay, ca_depth]
    
    # FROZEN PARAMETERS are passive parameters fit previously in passive model
    # NOTE: leave empty if you want to use passive params determined by folding method
    self.frozen_params = [] # [dend_gl_param, dend_cm_param] # SETPARAM: frozen params from previous optimisations

    # FREE PARAMETERS are active conductances with large impact on response
    # SETPARAM: parameters that are optimised (must be not frozen)
    self.free_params = (soma_gbar_params + [soma_gl_factor, soma_cm_factor] + 
                        dend_gbar_params + [dend_gl_factor, dend_cm_factor])
    
    # List of all used ephys.parameters
    self.used_params = self.frozen_params + self.free_params
    for param in self.frozen_params: assert param.frozen
    for param in self.free_params: assert (not param.frozen)

# RUN CELL
make_parameters(opt)

## Protocols for optimisation

In [ ]:
def make_model_protocols(self):
    """
    Make Cell Model and stimulation protocols
    
    @note    must be called only once in incremental optimisation
    """
    # Choose model we want to optimize
    fold_method = 'marasco'
    num_fold_passes = 7

    red_model = StnReducedModel(
                    name        = 'StnFolded',
                    fold_method = fold_method,
                    num_passes  = num_fold_passes)

    # Protocols to use for optimisation
    opt_stim_protocols = [MIN_SYN_BURST, SYN_BACKGROUND_HIGH] # SETPARAM: stimulation protocols for evaluation of fitness
    self.proto_kwargs = { # SETPARAM: extra parameters for protocols
        SYN_BACKGROUND_HIGH: {
            'num_syn_gpe': 12,
        }
    }
    
    # Make all protocol data
    red_protos = {
        stim_proto: BpopProtocolWrapper.make(stim_proto, **self.proto_kwargs[stim_proto]) 
            for stim_proto in opt_stim_protocols
    }

    # Collect al frozen mechanisms and parameters required for protocols to work
    proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(red_protos.values())

    # Distinguish between sets of parameters (used, frozen, free/optimised)
    self.frozen_params += proto_params
    self.used_params = self.frozen_params + self.free_params
    for param in self.frozen_params: assert param.frozen
    for param in self.free_params: assert (not param.frozen)

    # Assign parameters to reduced model
    red_model.set_mechs(proto_mechs)
    red_model.set_params(self.used_params)
    
    # Update optimisation data
    self.opt_stim_protocols = opt_stim_protocols
    self.red_model = red_model
    self.red_protos_wrappers = red_protos
    self.red_ephys_protos = [w.ephys_protocol for w in self.red_protos_wrappers.values()]
    
# RUN CELL
make_model_protocols(opt)

## Make Features & Target Values

In [ ]:
def make_efeatures_targets(self):
    """
    Make EFeature objects for each stimulation protocol, and calculate target values from
    full model responses.
    
    @note    must be called only once in incremental optimisation
    """
    # Make EFEL feature objects : dict(StimProtocol : dict(feature_name : tuple(efeature, weight)))
    stimprotos_feats = features_stn.make_opt_features(self.red_protos_wrappers.values())
    
    # Convert to list(efeature) and list(weight)
    all_opt_features, all_opt_weights = features_stn.all_features_weights(stimprotos_feats.values())
    
    ##################################################################################################
    # 1. SET EFEATURE.EXP_MEAN FROM FULL MODEL FEATURE VALUE
    
    full_proto_wrappers = [
        BpopProtocolWrapper.make(stim_proto, **self.proto_kwargs[stim_proto]) 
            for stim_proto in self.red_protos_wrappers.keys()
    ]
    full_mechs, full_params = BpopProtocolWrapper.all_mechs_params(full_proto_wrappers)
    full_model = StnFullModel(
                    name		= 'StnGillies',
                    mechs		= full_mechs,
                    params		= full_params)

    self.full_responses = run_proto_responses(
                        full_model,
                        [p.ephys_protocol for p in full_proto_wrappers])
    
    plot_proto_responses(self.full_responses)
    
    # Calculate feature values from responses (assigns efeature.exp_mean)
    features_stn.calc_feature_targets(stimprotos_feats, self.full_responses)
    
    # Update optimisation data
    self.stimprotos_feats = stimprotos_feats
    self.all_opt_features = all_opt_features
    
# RUN CELL
make_efeatures_targets(opt)

## Objectives, Fitness Calculator, Cell Evaluator

In [ ]:
def make_costfunc(self):
    """
    Make everything for evaluating the cost function on a particular cell model, i.e.
    ephys.objectives, ephys.objectivescalculators, ephys.evaluators.
    """

    # NOTE: effective weights are set using 'exp_std' of the EFeature objects
    all_objectives = [ephys.objectives.SingletonObjective(f.name, f) for f in self.all_opt_features]

    # Calculator maps model responses to scores
    fitcalc = ephys.objectivescalculators.ObjectivesCalculator(all_objectives)

    # Make evaluator to evaluate model using objective calculator
    opt_ephys_protos = {k.name: v.ephys_protocol for k,v in self.red_protos_wrappers.iteritems()}
    opt_params_names = [param.name for param in self.free_params]

    nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

    cell_evaluator = ephys.evaluators.CellEvaluator(
                        cell_model			= self.red_model,
                        param_names			= opt_params_names, # fitted parameters
                        fitness_protocols	= opt_ephys_protos,
                        fitness_calculator	= fitcalc,
                        sim					= nrnsim,
                        isolate_protocols	= True)
    
    # Update optimisation data
    self.fitcalc = fitcalc
    self.cell_evaluator = cell_evaluator
    
# RUN CELL
make_costfunc(opt)

## Optimisation

In [ ]:
def make_optimisation(self, checkpoints_file=None):
    # Optimisation parameters
    population_size = 100 # SETPARAM: population size
    deap_seed = 8 # SETPARAM: seed used by evolutionary algorithm (for population variability)
    parallel = True # SETPARAM: parallel execution using ipyparallel
    map_function = get_map_function(parallel)

    # Checkpoints: for each generation save [population, generation, parents, halloffame, history, logbook, rndstate]
    self.continued_from_cp = (checkpoints_file is not None)
    if not self.continued_from_cp:
        import uuid
        uuid_head = str(uuid.uuid1())[0:8]
        data_dir = '/home/luye/cloudstore_m/simdata/marasco_folding' # SETPARAM: data save location
        checkpoints_file = data_dir + '/opt_checkpoints_' + uuid_head + '.pkl'
    
    print('Optimisation checkpoints will be saved to file:\n{}'.format(checkpoints_file))
    
    # Make optimisation using the model evaluator
    optimisation = bpop_optimisation.DEAPOptimisation(
                        evaluator		= self.cell_evaluator,
                        offspring_size	= population_size,
                        map_function    = map_function,
                        selector_name   = 'IBEA', # SETPARAM: selector operator (default 'IBEA')
                        mutpb           = 1.0,     # SETPARAM: mutation rate (default 1.0)
                        cxpb            = 1.0,     # SETPARAM: crossover rate (default 1.0)
                        seed            = deap_seed)
    
    # Update optimisation data
    self.checkpoints_file = checkpoints_file
    self.population_size = population_size
    self.optimisation = optimisation
    
# RUN CELL
continue_cp = None # SETPARAM: checkpoints file to continue from
make_optimisation(opt, continue_cp)

In [ ]:
def run_optimisation(self):
    """
    Run incremental optimization: run for fixed number of generations, 
    increase number of folding passes, repeat
    """
    
    num_folds_per_step = [7] # SETPARAM: number of folding steps [1, 2, 3, 5, 7]
    num_generations = 100 # SETPARAM: number of generations per step (MINIMUM 2 or error)
    tot_generations = 0 # number of generations run so far
    self.checkpoints = []
    
    for i_step, num_folds in enumerate(num_folds_per_step):
        
        # Increment number of folding passes
        self.red_model._num_passes = num_folds
        
        # Optimize new reduced model
        print("""
        #######################################################################################
        # Starting optimization of reduced model with {} folding passes
        #######################################################################################
        """.format(num_folds))
        
        opt_outputs = self.optimisation.run(
                        max_ngen    = tot_generations + num_generations,
                        cp_filename = self.checkpoints_file,
                        continue_cp = (i_step != 0) or self.continued_from_cp)
        
        # NOTE: opt_outputs saved to pickle already, may become very large
                                
        tot_generations += num_generations
        
# RUN CELL
run_optimisation(opt)

## Save optimisation data

In [ ]:
def save_optimisation_settings(self):
    """
    Append additional optimization info/settings to checkpoints file.
    
    Saved information includes frozen and free parameter names, EFeatures making up
    the cost/fitness function and their target values and weights, seeds for random
    number generators.
    """
    proto_feat_info = {}
    
    # Create dict {StimProtol: {feat_name : {exp_mean/std : value } } }
    for stim_proto, feat_dict in self.stimprotos_feats.iteritems():

        proto_feat_info[stim_proto.name] = {

            feat_name: {
                'weight': feat_data[1], 
                'exp_mean': feat_data[0].exp_mean, 
                'exp_std': feat_data[0].exp_std
            } 

            for feat_name, feat_data in feat_dict.iteritems()
        }

    # Save module info
    import sys
    code_version_info = {}
    modulenames = set(sys.modules) & set(('bluepyopt', 'efel', 'elephant')) # & set(globals())
    for module_name in modulenames:
        code_version_info[module_name] = getattr(sys.modules[module_name], '__version__', 'unknown')
    
    # Our code version
    head_SHA1 = %sx git log -1 --format="%H"
    code_version_info['bgcellmodels'] = head_SHA1
    
    # Put everything in dict for pickling
    info_opt = {
        'opt_param_names': self.cell_evaluator.param_names, # in same order as individual params
        'free_params': {p.name: p for p in self.free_params},
        'frozen_params': {p.name: p for p in self.frozen_params},
        'objectives_ordered': {o.name: o for o in self.cell_evaluator.fitness_calculator.objectives},
        'deap_seed': self.optimisation.seed,
        'protos_feats': proto_feat_info,
        'code_version_info': code_version_info,
    }

    # pp.pprint(info_opt)
    self.settings_file = self.checkpoints_file[:-4] + '_settings.pkl'
    with open(self.settings_file, 'ab') as f: # appends to file stream, load using second 'pickle.load()'
        pickle.dump(info_opt, f)
        
    print("Following optimisation settings saved to pickle file {} :".format(self.settings_file))
    pp.pprint(info_opt)
        
# RUN CELL
save_optimisation_settings(opt)

## Analyze optimization results

We want to know how our individuals (candidate parameter sets) evolved during the optimization, and potentially how their responses look.

In [ ]:
# OPTIONAL: reload previous results from checkpoints file
# checkpoints_file = '/home/luye/cloudstore_m/simdata/marasco_folding/opt_checkpoints_b58364be.pkl'
checkpoints_file = opt.checkpoints_file

# Old pickling method
# with open(checkpoints_file, 'r') as f:
#     checkpoint = pickle.load(f)
#     # old_param_names = pickle.load(f)

# New pickling method
import cPickle
with open(checkpoints_file, "rb") as f:
    while True:
        try:
            checkpoint = cPickle.load(f)
        except EOFError:
            break

# Get optimisation logs
hall_of_fame = checkpoint['halloffame']
logs = checkpoint['logbook']
pareto_front = checkpoint['paretofront']

In [ ]:
# Plot evolution of fitness values
fig, ax = pop_analysis.plot_log(logs)

## Evaluate / Validate optimization results

We want to know how our optimized model performs under other stimulation protocols that were not part of the objective function. This is a form of cross-validation.

In [ ]:
ephys_protocols = opt.optimisation.evaluator.fitness_protocols

### Evaluate best candidate

In [ ]:
print("\nSum of squared error for best individuals:")
costs = np.array([-ind.fitness.neg_squared_sum for ind in hall_of_fame])
print(costs.reshape((min(costs.size, 10),-1), order='F'))

In [ ]:
# Print fittest individual (best parameter set)
best_params_dict = opt.optimisation.evaluator.param_dict(hall_of_fame[0])
print('Fittest individual:')
pp.pprint(best_params_dict)

# opt.cell_evaluator.evaluate_with_dicts(best_params_dict)
logutils.setLogLevel('quiet', ['stn_protos', 'bpop_ext', 'reducemodel.reduce_cell', 'bluepyopt.ephys.efeatures'])

# red_responses = {
#     proto.name : proto.run(
#                     cell_model		= opt.red_model, 
#                     param_values	= best_params_dict,
#                     sim				= nrnsim,
#                     isolate			= True)
#     for proto in opt.red_ephys_protos
# }

red_responses = opt.cell_evaluator.run_protocols(
                    opt.red_ephys_protos,
                    best_params_dict)

scores = opt.fitcalc.calculate_scores(red_responses)

pp.pprint(scores)
print('\nSum of efeature scores = {}'.format(sum(scores.values())))
print('\nMin of efeature scores = {}'.format(min(scores.values())))

resp_analysis.plot_responses(red_responses)

### Validation: compare non-optimized responses

In [ ]:
%%capture

# Plot reduced model responses (to all available stimulation protocols)
all_stim_protos = protocols_stn.PROTOCOL_WRAPPERS.keys()
red_proto_wrappers = [
    BpopProtocolWrapper.make(stim_proto, StnModel.Gillies_FoldMarasco) 
        for stim_proto in all_stim_protos
]

# Make ephys parameters for all protocols
ephys_protos = [p.ephys_protocol for p in red_proto_wrappers]
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(red_proto_wrappers)

# Make model that includes all parameters
red_model = StnReducedModel( # make model version with all protocol mechanisms
                name        = 'StnFolded',
                fold_method = 'marasco',
                num_passes  = 7,
                mechs       = proto_mechs,
                params      = proto_params + opt.free_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

red_responses = {
    proto.name : proto.run(
                    cell_model		= red_model, 
                    param_values	= best_params_dict,
                    sim				= nrnsim,
                    isolate			= True)
    for proto in ephys_protos
}

In [ ]:
resp_analysis.plot_proto_responses(red_responses)

In [ ]:
%%capture

all_stim_protos = protocols_stn.PROTOCOL_WRAPPERS.keys()
full_proto_wrappers = [
    BpopProtocolWrapper.make(stim_proto, StnModel.Gillies2005) 
        for stim_proto in all_stim_protos
]
full_ephys_protos = [p.ephys_protocol for p in full_proto_wrappers]
full_mechs, full_params = BpopProtocolWrapper.all_mechs_params(full_proto_wrappers)

full_model = StnFullModel(
                name		= 'StnGillies',
                mechs		= full_mechs,
                params		= full_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

full_responses = {
    proto.name : proto.run(
                    cell_model		= full_model, 
                    param_values	= {},
                    sim				= nrnsim,
                    isolate			= True)
    for proto in full_ephys_protos
}

In [ ]:
resp_analysis.plot_proto_responses(full_responses)